# ENSF 444 Project Group 31



### Step 0: Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore') #ignoring some deprication warnings

### Step 1: Data Input

The data used for this task is the TMDB 5000 Movie Dataset on Kaggle

In [3]:
data = pd.read_csv('tmdb_5000_movies.csv')
data.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995.0,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2.787965e+09,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800.0
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285.0,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,9.610000e+08,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500.0
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647.0,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,8.806746e+08,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466.0
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026.0,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1.084939e+09,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106.0
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529.0,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,2.841391e+08,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124.0


### Step 2: Data Processing

In [4]:
print(data.isnull().sum()) #checking for null values in the dataset

budget                     0
genres                     9
homepage                3103
id                         9
keywords                   9
original_language          9
original_title             9
overview                  12
popularity                 9
production_companies       9
production_countries       9
release_date              10
revenue                    9
runtime                   11
spoken_languages           9
status                     9
tagline                  854
title                      9
vote_average               9
vote_count                 9
dtype: int64


In [ ]:
# Columns with minimal missing values (<20) that we want to keep:
# For numerical columns, fill with median (less sensitive to outliers)
data['runtime'].fillna(data['runtime'].median(), inplace=True)
data['release_date'].fillna(data['release_date'].mode()[0], inplace=True)  # Fill with most common date

# For text columns, fill with empty string
data['overview'].fillna('', inplace=True)

# Drop columns with many missing values or not relevant for popularity prediction
data.drop(['homepage', 'tagline', 'keywords', 'production_companies', 
           'production_countries', 'spoken_languages'], axis=1, inplace=True)

# Drop remaining rows with missing values
data.dropna(inplace=True)

# Verify no more missing values
print("\nMissing values after cleaning:")
print(data.isnull().sum())


Missing values after cleaning:
budget               0
genres               0
id                   0
original_language    0
original_title       0
overview             0
popularity           0
release_date         0
revenue              0
runtime              0
status               0
title                0
vote_average         0
vote_count           0
dtype: int64


#### Parse Data for JSON Columns

In [6]:
def parse_json(json_str):
    try:
        return [item['name'] for item in ast.literal_eval(json_str)]
    except:
        return []

data['genres_list'] = data['genres'].apply(parse_json)

#### Parse JSON columns

In [7]:
import ast

def parse_json(json_str):
    try:
        return [item['name'] for item in ast.literal_eval(json_str)]
    except:
        return []

data['genres_list'] = data['genres'].apply(parse_json)

# Create binary columns for top genres
top_genres = ['Action', 'Adventure', 'Animation', 'Comedy', 'Drama', 
              'Horror', 'Romance', 'Science Fiction', 'Thriller']
for genre in top_genres:
    data[genre] = data['genres_list'].apply(lambda x: 1 if genre in x else 0)

# Convert release_date to datetime and extract year/month features
data['release_date'] = pd.to_datetime(data['release_date'])
data['release_year'] = data['release_date'].dt.year
data['release_month'] = data['release_date'].dt.month

# Select features
features = ['budget', 'runtime', 'release_year', 'release_month'] + top_genres
target = 'popularity'

#### Preprocessing Setup

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer

# After cleaning your data as previously discussed, define preprocessing:
preprocessor = ColumnTransformer(
    transformers=[
        # Numeric columns (scale them)                              
        ('num', StandardScaler(), ['budget', 'runtime', 'release_year', 'release_month']),      ## vote_count -> release_year, release_month
        
        # Categorical columns (one-hot encode)
        #('cat', OneHotEncoder(handle_unknown='ignore'), 
         #['original_language', 'production_country']),
        
        # Text data from genres (if kept as text)
        #('text', CountVectorizer(), 'genres_combined')  # If you combine genre lists into text
    ],
    remainder='passthrough'  # Keeps other columns unchanged
)

#### Separate data into feature matrix and target vector

In [9]:
X = data[features]
y = data[target]

### Step 3: Implement Machine Learning Models

In [13]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

#### Model 1: Linear Regression

#### Model 3: SVM

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV, cross_val_score


# Create SVM pipeline
svm_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR()) 
])

# Implement grid search
param_grid = {
    'regressor__C': [0.1, 1, 10, 100],
    'regressor__gamma': ['scale', 'auto', 0.01, 0.1],
    'regressor__kernel': ['rbf', 'linear']
}

grid_search = GridSearchCV(
    estimator=svm_pipeline,
    param_grid=param_grid,
    scoring='r2',               
    cv=5,
    n_jobs=-1,
    return_train_score=True
)

# Fit to training data
grid_search.fit(X_train, y_train)

best_idx = grid_search.best_index_

# Print results
print("Best params:\n{}\n".format(grid_search.best_params_))
print("Best cross-validation train R^2: {:.2f}".format(grid_search.cv_results_['mean_train_score'][best_idx]))  # Training R²
print("Best cross-validation R^2: {:.2f}".format(grid_search.best_score_))  # Validation R²
print("Test-set R^2: {:.2f}".format(grid_search.score(X_test, y_test)))  # Test R²

Best params:
{'regressor__C': 100, 'regressor__gamma': 'auto', 'regressor__kernel': 'rbf'}

Best cross-validation train R^2: 0.37
Best cross-validation R^2: 0.35
Test-set R^2: 0.25
